# Imports

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import pandas as pd

import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
from textblob import TextBlob
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report
import ast
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from NseXRBL import *
from Preprocessing import *
from Sentiments import *

# 1) Data Extraction

-------------------------------------------------------------------------------------------------------------------

### Getting quarterly financial data fron NSE (National stock exchange, India)

![Alt Text](Images/Image1.png)


In [2]:
file_path = 'Data/CF-FR-equities-ZOMATO-24-Mar-2025.csv'

df_fillings = pd.read_csv(file_path)
df_fillings.head(3)

,COMPANY NAME,AUDITED / UNAUDITED,CUMULATIVE / NON-CUMULATIVE,CONSOLIDATED / NON-CONSOLIDATED,IND AS/ NON IND AS,PERIOD,PERIOD ENDED,RELATING TO,** XBRL,Exchange Received Time,Exchange Dissemination Time,Time Taken
0,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Dec-2024,Third Quarter,https://nsearchives.nseindia.com/corporate/xbr...,20-Jan-2025 18:29:47,20-Jan-2025 18:30:33,00:00:46
1,Zomato Limited,Un-Audited,Non-cumulative,Consolidated,Ind-AS New,Quarterly,31-Dec-2024,Third Quarter,https://nsearchives.nseindia.com/corporate/xbr...,20-Jan-2025 18:28:59,20-Jan-2025 18:29:27,00:00:28
2,Zomato Limited,Un-Audited,Non-cumulative,Consolidated,Ind-AS New,Quarterly,30-Sep-2024,Second Quarter,https://nsearchives.nseindia.com/corporate/xbr...,22-Oct-2024 19:01:05,22-Oct-2024 19:01:37,00:00:32


Let's understand this table:- 

- COMPANY NAME : Name of the company

- AUDITED / UNAUDITED : Weather the report sent was audited or not

- CUMULATIVE / NON-CUMULATIVE : The report is Cumalative or not

- CONSOLIDATED / NON-CONSOLIDATED : The company releases 2 reports at same time, consolidated or non-consolidated. The consolidated report will have profits and expense of previous quarters included. where as non-consolidated report will have profits and expense of the particular quarter only (which we want in our case)

- IND AS/ NON IND AS : Its represent the accounting standards used in the report

- PERIOD ENDED : End date of quarter 

- RELATING TO : The report belongs to what quarter (First, Second, Third, )

- ** XBRL : XML link of the the report(in XRBL format)

- Exchange Received Time : At what time the exchange recieved report from the company

- Exchange Dissemination Time : At what time exchange published the report 

- Time Taken : difference between 'Exchange Dissemination Time' and 'Exchange Received Time'

applying preprocessing to clean the data

In [3]:
df_fillings = preprocess_fillings_NonConsolidated_csv(file_path)
df_fillings.head()

,COMPANY NAME,AUDITED / UNAUDITED,CUMULATIVE / NON-CUMULATIVE,CONSOLIDATED / NON-CONSOLIDATED,IND AS/ NON IND AS,PERIOD,PERIOD ENDED,RELATING TO,** XBRL,Exchange Received Time,Exchange Dissemination Time,Time Taken
0,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Dec-2024,Third Quarter,https://nsearchives.nseindia.com/corporate/xbr...,20-Jan-2025 18:29:47,20-Jan-2025 18:30:33,00:00:46
3,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,30-Sep-2024,Second Quarter,https://nsearchives.nseindia.com/corporate/xbr...,22-Oct-2024 19:00:01,22-Oct-2024 19:00:31,00:00:30
5,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,30-Jun-2024,First Quarter,https://nsearchives.nseindia.com/corporate/xbr...,01-Aug-2024 18:59:42,01-Aug-2024 18:59:58,00:00:16
7,Zomato Limited,Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Mar-2024,Fourth Quarter,https://nsearchives.nseindia.com/corporate/xbr...,13-May-2024 21:26:06,13-May-2024 21:26:13,00:00:07
9,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Dec-2023,Third Quarter,https://nsearchives.nseindia.com/corporate/xbr...,08-Feb-2024 18:33:31,08-Feb-2024 18:33:31,::


In [4]:
# Testing
Xml_Extract_Financials(df_fillings['** XBRL'].iloc[0])

,FullTag,Value
0,RevenueFromOperations,22260000000.00
1,OtherIncome,3110000000.00
2,Income,25370000000.00
3,CostOfMaterialsConsumed,0.00
4,PurchasesOfStockInTrade,0.00
5,ChangesInInventoriesOfFinishedGoodsWorkInProgr...,0.00
6,EmployeeBenefitExpense,3330000000.00
7,FinanceCosts,40000000.00
8,DepreciationDepletionAndAmortisationExpense,270000000.00
9,OtherExpenses,15990000000.00


Here we are successfully able to extract financial report from the xml link

But from this we want only few important values like, Income, Revenue, Expenses etc.

In [5]:
# Testing
Extract_Important_Financials(df_fillings['** XBRL'].iloc[0])

,Total_Revenue,RevenueFromOperations,ProfitBeforeTax,FinanceCosts,DepreciationDepletionAndAmortisationExpense,Total_Expenses
0,2.537000e+10,2.226000e+10,5.740000e+09,40000000.0,270000000.0,1.963000e+10


My functions are working fine, so let's apply it to all rows

In [6]:
new_financial_data = df_fillings['** XBRL'].apply(Extract_Important_Financials)
# Concatenate results into a DataFrame
financial_df = pd.concat(new_financial_data.tolist(), axis=0).reset_index(drop=True)

# Concatenate with original df_fillings
financial_df = pd.concat([df_fillings.reset_index(drop=True), financial_df], axis=1)

financial_df.head()

,COMPANY NAME,AUDITED / UNAUDITED,CUMULATIVE / NON-CUMULATIVE,CONSOLIDATED / NON-CONSOLIDATED,IND AS/ NON IND AS,PERIOD,PERIOD ENDED,RELATING TO,** XBRL,Exchange Received Time,Exchange Dissemination Time,Time Taken,Total_Revenue,RevenueFromOperations,ProfitBeforeTax,FinanceCosts,DepreciationDepletionAndAmortisationExpense,Total_Expenses
0,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Dec-2024,Third Quarter,https://nsearchives.nseindia.com/corporate/xbr...,20-Jan-2025 18:29:47,20-Jan-2025 18:30:33,00:00:46,2.537000e+10,2.226000e+10,5.740000e+09,40000000.0,270000000.0,1.963000e+10
1,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,30-Sep-2024,Second Quarter,https://nsearchives.nseindia.com/corporate/xbr...,22-Oct-2024 19:00:01,22-Oct-2024 19:00:31,00:00:30,2.420000e+10,2.151000e+10,4.970000e+09,40000000.0,220000000.0,1.923000e+10
2,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,30-Jun-2024,First Quarter,https://nsearchives.nseindia.com/corporate/xbr...,01-Aug-2024 18:59:42,01-Aug-2024 18:59:58,00:00:16,2.327000e+10,2.048000e+10,4.700000e+09,40000000.0,190000000.0,1.854000e+10
3,Zomato Limited,Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Mar-2024,Fourth Quarter,https://nsearchives.nseindia.com/corporate/xbr...,13-May-2024 21:26:06,13-May-2024 21:26:13,00:00:07,2.093000e+10,1.824000e+10,3.960000e+09,40000000.0,190000000.0,1.658000e+10
4,Zomato Limited,Un-Audited,Non-cumulative,Non-Consolidated,Ind-AS New,Quarterly,31-Dec-2023,Third Quarter,https://nsearchives.nseindia.com/corporate/xbr...,08-Feb-2024 18:33:31,08-Feb-2024 18:33:31,::,2.018000e+10,1.782000e+10,3.850000e+09,50000000.0,190000000.0,1.633000e+10


In [7]:
# checking for null values
financial_df.isnull().sum()

COMPANY NAME                                   0
AUDITED / UNAUDITED                            0
CUMULATIVE / NON-CUMULATIVE                    0
CONSOLIDATED / NON-CONSOLIDATED                0
IND AS/ NON IND AS                             0
PERIOD                                         0
PERIOD ENDED                                   0
RELATING TO                                    0
** XBRL                                        0
Exchange Received Time                         0
Exchange Dissemination Time                    0
Time Taken                                     0
Total_Revenue                                  0
RevenueFromOperations                          0
ProfitBeforeTax                                0
FinanceCosts                                   0
DepreciationDepletionAndAmortisationExpense    0
Total_Expenses                                 0
dtype: int64

In [8]:
# saving results
financial_df.to_excel("Data/Zomato_Financials.xlsx", index=False)

-------------------------------------------------------------------------------------------------------------------

### Getting historical stock price data

In [10]:
stock_name = 'Zomato' 
market = '.NS' # For NSE, Indian market
stock_price_file_path = f"./Data/{stock_name}.csv"

Checking if the data file already exists or not

If the file does not exist, download the complete data

In [11]:
stock_price_df = preprocess_stock_price_data(stock_name, market, stock_price_file_path)
stock_price_df.sort_values('Date', ascending=False).head()

The CSV file is already up to date.


,Date,Adj Close,Close,High,Low,Open,Volume
907,2025-03-24,NaN,222.800003,230.199997,222.199997,228.800003,41909431.0
906,2025-03-21,NaN,227.520004,228.850006,220.000000,220.000000,113328549.0
905,2025-03-20,NaN,224.470001,228.000000,221.869995,226.000000,61376776.0
904,2025-03-19,NaN,223.529999,227.500000,218.500000,218.500000,65519150.0
903,2025-03-18,NaN,203.690002,203.690002,203.690002,203.690002,0.0


-------------------------------------------------------------------------------------------------------------------

### Getting Headlines Data for Sentimental Analysis

In [12]:
sentiment_file_path = f'Data/{stock_name}_sentiments.csv'
headlines_df = preprocess_sentiment_file(stock_name, market, sentiment_file_path)
headlines_df.sort_values('Date', ascending=False).head()

The CSV file is already up to date.


,Date,Articles,Aggregated Sentiment Score
1340,2025-03-24,[Zomato Share Price Highlights : Zomato closed...,NaN
1339,2025-03-23,"[Zomato, Kondapur restaurant fined for deliver...",0.000000
1338,2025-03-22,[Zomato Renamed as Eternal Limited – Effective...,0.106506
1337,2025-03-21,"[Zomato Share Price Today Live Updates, 21 Mar...",-0.027511
1336,2025-03-20,"[Zomato Closing Price Settles Near Opening, Fa...",0.004665


Using Google News to fetch latest market news of the company by web scrapping.

In this section I will find all the news of the company since it has been listed in stock market.

In [15]:
# Find the rows where 'Articles' is empty
empty_articles_rows = headlines_df[headlines_df['Articles'].isnull()]

headlines_df.loc[empty_articles_rows.index, 'Articles'] = empty_articles_rows['Date'].apply(lambda date: get_google_news(stock_name, date))

headlines_df.sort_values('Date', ascending=False).head()


,Date,Articles,Aggregated Sentiment Score
1340,2025-03-24,[Zomato Share Price Highlights : Zomato closed...,NaN
1339,2025-03-23,"[Zomato, Kondapur restaurant fined for deliver...",0.000000
1338,2025-03-22,[Zomato Renamed as Eternal Limited – Effective...,0.106506
1337,2025-03-21,"[Zomato Share Price Today Live Updates, 21 Mar...",-0.027511
1336,2025-03-20,"[Zomato Closing Price Settles Near Opening, Fa...",0.004665


In [15]:
# Save progress
headlines_df.to_csv(sentiment_file_path, index= False)

Now, we will mark each article with a sentiment score. 

Interpretation of Sentiment Score:
Range: Sentiment scores typically range from -1 to +1.
Negative values (below 0): Negative sentiment.
Positive values (above 0): Positive sentiment.
Around 0: Neutral or mixed sentiment.

Since, there could be multiple articles from different sources about the company, which shows different opinions of people, we will aggregate the sentiment score for each day.

In [16]:
# Rows where Aggregated Sentiment Score is empty
empty_score_rows = headlines_df[headlines_df['Aggregated Sentiment Score'].isnull() & headlines_df['Articles'].notnull()]

# Apply the function to calculate sentiment score
headlines_df.loc[empty_score_rows.index,'Aggregated Sentiment Score'] = empty_score_rows['Articles'].apply(get_aggregated_sentiment)

# Display the updated DataFrame
headlines_df.sort_values('Date', ascending=False).head()

Negative:- Zomato Share Price Highlights : Zomato closed today at ₹222.70, down -2.13% from yesterday's ₹227.55 | Stock Market News - Mint
Positve:- Zomato Most Preferred Food Delivery App, Swiggy Bolt Ahead In 10-15 Minute Segment: BofA Survey - NDTV Profit
Neutral:- Sahibjeet Singh Sawhney: Leading brand communication to connect Zomato with millions - Exchange4Media
Positve:- “Ek, Do, Teen…Dus!” Zomato’s New Campaign Highlights Quick 10-Minute Delivery - Passionate In Marketing


,Date,Articles,Aggregated Sentiment Score
1340,2025-03-24,[Zomato Share Price Highlights : Zomato closed...,0.050704
1339,2025-03-23,"[Zomato, Kondapur restaurant fined for deliver...",0.000000
1338,2025-03-22,[Zomato Renamed as Eternal Limited – Effective...,0.106506
1337,2025-03-21,"[Zomato Share Price Today Live Updates, 21 Mar...",-0.027511
1336,2025-03-20,"[Zomato Closing Price Settles Near Opening, Fa...",0.004665


Here, I was able to successfully score each article. And then was able to find its aggregated Sentiment score.

eg, 
What 0.0759 Means:
A score of 0.0759 is slightly positive, indicating that the overall sentiment of the article titles in that row is mildly optimistic or slightly favorable. However, it is not a strong positive sentiment. This suggests:

The articles may contain neutral to mildly positive language.
The content could be balanced, with both positive and negative aspects discussed, but with a slight lean towards positivity.

In [18]:
# Save progress
headlines_df.to_csv(sentiment_file_path, index= False)

Future scope for optimization...

"*To add this feature into our model, which should aggreate it further more considering the market holidays.* 
*Eg, A company news came on saturday, sunday and monday, will have a impact on its stock on monday. or holidays like chritsmas etc.*"

-------------------------------------------------------------------------------------------------------------------

# 2) Data Modeling & Feature Engineering

In [ ]:
# Saving results
#financial_df['Date'] = pd.to_datetime(df_combined['Exchange Dissemination Time'], 
#                                                            format= '%d-%b-%Y %H:%M:%S').dt.strftime('%Y-%m-%d')



In [552]:
model_data = pd.merge(stock_data, sentiment_score, how='left', on='Date')
model_data.head()

,Date,Adj Close,Close,High,Low,Open,Volume,Articles,Aggregated Sentiment Score
0,2021-07-23,126.000000,126.000000,138.899994,115.000000,116.000000,694895290.0,[Zomato IPO catapults founder towards list of ...,0.062067
1,2021-07-26,140.649994,140.649994,143.750000,125.300003,126.349998,249723854.0,[Zomato share gains for second straight sessio...,0.080000
2,2021-07-27,132.899994,132.899994,147.800003,127.750000,141.699997,240341900.0,[Inside Zomato and Universal Music Group's rec...,0.058333
3,2021-07-28,131.199997,131.199997,135.000000,123.550003,131.000000,159793731.0,"[Sanjeev Bikhchandani: Zomato a validation, pu...",0.071250
4,2021-07-29,141.550003,141.550003,144.000000,132.199997,134.949997,117973089.0,[],0.000000


In [553]:

model_data['Aggregated Sentiment Score'] = model_data['Aggregated Sentiment Score'].replace(0, None).ffill()
model_data.drop(['Articles', 'Adj Close'], axis=1, inplace=True)
model_data.head()

,Date,Close,High,Low,Open,Volume,Aggregated Sentiment Score
0,2021-07-23,126.000000,138.899994,115.000000,116.000000,694895290.0,0.062067
1,2021-07-26,140.649994,143.750000,125.300003,126.349998,249723854.0,0.080000
2,2021-07-27,132.899994,147.800003,127.750000,141.699997,240341900.0,0.058333
3,2021-07-28,131.199997,135.000000,123.550003,131.000000,159793731.0,0.071250
4,2021-07-29,141.550003,144.000000,132.199997,134.949997,117973089.0,0.071250


In [554]:
Zomato_Financials_df['Date'] = pd.to_datetime(Zomato_Financials_df['Date'])

model_data = pd.merge(model_data, Zomato_Financials_df, how='left', on= 'Date')
model_data.head()

,Date,Close,High,Low,Open,Volume,Aggregated Sentiment Score,Total_Revenue,RevenueFromOperations,ProfitBeforeTax,FinanceCosts,DepreciationDepletionAndAmortisationExpense,Total_Expenses
0,2021-07-23,126.000000,138.899994,115.000000,116.000000,694895290.0,0.062067,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-07-26,140.649994,143.750000,125.300003,126.349998,249723854.0,0.080000,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-07-27,132.899994,147.800003,127.750000,141.699997,240341900.0,0.058333,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-07-28,131.199997,135.000000,123.550003,131.000000,159793731.0,0.071250,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-07-29,141.550003,144.000000,132.199997,134.949997,117973089.0,0.071250,NaN,NaN,NaN,NaN,NaN,NaN


In [555]:
model_data[['Total_Revenue', 'RevenueFromOperations', 'ProfitBeforeTax',	'FinanceCosts',	'DepreciationDepletionAndAmortisationExpense',	'Total_Expenses']] = model_data[['Total_Revenue', 'RevenueFromOperations', 'ProfitBeforeTax',	'FinanceCosts',	'DepreciationDepletionAndAmortisationExpense',	'Total_Expenses']].fillna(method='ffill')
model_data.head()

,Date,Close,High,Low,Open,Volume,Aggregated Sentiment Score,Total_Revenue,RevenueFromOperations,ProfitBeforeTax,FinanceCosts,DepreciationDepletionAndAmortisationExpense,Total_Expenses
0,2021-07-23,126.000000,138.899994,115.000000,116.000000,694895290.0,0.062067,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-07-26,140.649994,143.750000,125.300003,126.349998,249723854.0,0.080000,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-07-27,132.899994,147.800003,127.750000,141.699997,240341900.0,0.058333,NaN,NaN,NaN,NaN,NaN,NaN
3,2021-07-28,131.199997,135.000000,123.550003,131.000000,159793731.0,0.071250,NaN,NaN,NaN,NaN,NaN,NaN
4,2021-07-29,141.550003,144.000000,132.199997,134.949997,117973089.0,0.071250,NaN,NaN,NaN,NaN,NaN,NaN


In [556]:
model_data.dropna(inplace=True)
model_data.head()

,Date,Close,High,Low,Open,Volume,Aggregated Sentiment Score,Total_Revenue,RevenueFromOperations,ProfitBeforeTax,FinanceCosts,DepreciationDepletionAndAmortisationExpense,Total_Expenses
13,2021-08-11,135.649994,138.750000,123.000000,123.000000,111702781.0,0.037747,8.283000e+09,7.579000e+09,-2.908000e+09,18000000.0,319000000.0,1.118700e+10
14,2021-08-12,135.449997,137.399994,132.050003,135.649994,51256670.0,0.037747,8.283000e+09,7.579000e+09,-2.908000e+09,18000000.0,319000000.0,1.118700e+10
15,2021-08-13,137.350006,139.750000,132.100006,133.850006,33674300.0,0.060000,8.283000e+09,7.579000e+09,-2.908000e+09,18000000.0,319000000.0,1.118700e+10
16,2021-08-16,134.949997,136.899994,132.250000,136.399994,20305361.0,-0.052222,8.283000e+09,7.579000e+09,-2.908000e+09,18000000.0,319000000.0,1.118700e+10
17,2021-08-17,132.500000,134.350006,130.600006,132.800003,15815187.0,-0.052222,8.283000e+09,7.579000e+09,-2.908000e+09,18000000.0,319000000.0,1.118700e+10
